# 讀取資料
情書大全: https://www.dgreetings.com/love_romance_cards/letters.html

In [2]:
import pandas as pd
Corpus = pd.read_excel("Love_letters.xlsx")

In [3]:
Corpus.head()

,ID,Text
0,1,"Dear Darling,\nI have always wanted to write y..."
1,2,"Darling,\nI want you to know that there's no o..."
2,3,"Dear,\nMy life is beautiful because of you my ..."
3,4,"Dear,\nIn life there are certain things that w..."
4,5,"To my forever person,\nI think we make the per..."


建立一個大的文本集

In [4]:
All_text = ""
for text in Corpus["Text"]:
    All_text += text

將字母轉為小寫，減少之後字元向量的維度

In [5]:
All_text = All_text.lower()

In [6]:
print(f"文本集前190字元:\n{All_text[:190]} ")

文本集前190字元:
dear darling,
i have always wanted to write you a love letter, though i don't know how to begin.
when i am with you, i am lost of words, my heart yearns for you.
you are the love of my life. 


In [7]:
print(f"文本集共{len(All_text)}個字元") 

文本集共16285個字元


# 準備訓練集
把文本集切割成許多片段，並由下一個字元當作標籤

In [8]:
segment = [] # 文本片段
next_char = [] # 該片段的下一個字元
segment_len = 50
last_segment_start_ith = len(All_text)-segment_len # 最後一個片段的
for i in range(0,last_segment_start_ith):
    segment.append(All_text[i:i+segment_len])
    next_char.append(All_text[i+segment_len])

由於我們共有16285個字元，每50個字元切割一片段，每個切割後只右移1個字元，所以理論上有16285-50 = 16235個片段

In [9]:
print("(樣本數)片段數:",len(segment),"標籤數:",len(next_char))

(樣本數)片段數: 16235 標籤數: 16235


In [10]:
import numpy as np
i = np.random.randint(len(segment))
print(f"第{i}筆訓練資料:\n{segment[i]}\n") 
print(f"第{i}筆標籤:\n{next_char[i]}") 

第1201筆訓練資料:
s,to my forever person,
i think we make the perfec

第1201筆標籤:
t


# 字元向量
把a,b,c,...,z以及一些標點符號做one-hot encoding

In [11]:
all_char = []
for i in range(len(All_text)):
    all_char.append(All_text[i])
all_unique_char = np.unique(all_char)

取得所有獨立的字元

In [12]:
print(f"獨立字元有{len(all_unique_char)}個\n{all_unique_char}")

獨立字元有39個
['\n' ' ' '!' "'" ',' '-' '.' ';' '?' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '–'
 '—' '’' '…']


sklearn的OneHotEncoder()只能吃2d array

In [13]:
all_unique_char = np.array(all_unique_char)
all_unique_char = all_unique_char.reshape((len(all_unique_char),1)) #將39個字元轉為2d array

segment = np.array(segment)
segment = segment.reshape((len(segment),1)) #將39個字元轉為2d array

next_char = np.array(next_char)
next_char = next_char.reshape((len(next_char),1)) #將39個字元轉為2d array

In [14]:
all_unique_char[:]

array([['\n'],
       [' '],
       ['!'],
       ["'"],
       [','],
       ['-'],
       ['.'],
       [';'],
       ['?'],
       ['a'],
       ['b'],
       ['c'],
       ['d'],
       ['e'],
       ['f'],
       ['g'],
       ['h'],
       ['i'],
       ['j'],
       ['k'],
       ['l'],
       ['m'],
       ['n'],
       ['o'],
       ['p'],
       ['q'],
       ['r'],
       ['s'],
       ['t'],
       ['u'],
       ['v'],
       ['w'],
       ['x'],
       ['y'],
       ['z'],
       ['–'],
       ['—'],
       ['’'],
       ['…']], dtype='<U1')

In [15]:
next_char

array([['l'],
       ['o'],
       ['v'],
       ...,
       ['v'],
       ['e'],
       ['.']], dtype='<U1')

In [16]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(all_unique_char)
y = ohe.transform(next_char).toarray() # label

每1個字元(或稱token)現在由1個39維向量表示，所以y變成16235 x 39 維的矩陣

In [17]:
np.array(y)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

比較麻煩的是如何將segment變成 16235x50x39 的矩陣

In [18]:
# 我希望1個片段變成 50x39維的東西
text = segment[16234]
text

array(['u.\nlove you forever and always.\n\nwith lots of love'],
      dtype='<U50')

In [19]:
X = []
for i in range(len(segment)):
    text = segment[i]
    all_char_in_text = []
    for j in range(segment_len):
        all_char_in_text.append(text[0][j])
    all_char_in_text = np.array(all_char_in_text)
    all_char_in_text = all_char_in_text.reshape((-1,1))
    all_char_in_text = ohe.transform(all_char_in_text).toarray()
    #print(all_char_in_text.shape)
    X.append(all_char_in_text)

In [20]:
X = np.array(X)

趕緊看看上面的魔法成功了沒

In [21]:
# X.reshape((len(segment),segment_len,len(all_unique_char)))
X.shape

(16235, 50, 39)

In [22]:
X[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

# 建立LSTM模型

In [23]:
segment_len = 50 #1個片段有50個字元
vocab = 39 #有39個字元

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128,return_sequences= True,input_shape= (segment_len, vocab)))
model.add(LSTM(128,input_shape= (segment_len, vocab)))
model.add(Dense(vocab, activation= "softmax"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 50, 128)           86016     
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_8 (Dense)              (None, 39)                5031      
Total params: 222,631
Trainable params: 222,631
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])

In [110]:
model.fit(X, y, batch_size= 128, epochs= 128)

Train on 16235 samples
Epoch 1/128
16235/16235 [==============================] - 24s 1ms/sample - loss: 3.0099 - accuracy: 0.1826
Epoch 2/128
16235/16235 [==============================] - 20s 1ms/sample - loss: 2.8739 - accuracy: 0.2075s - loss: 2.8791 - accura
Epoch 3/128
16235/16235 [==============================] - 20s 1ms/sample - loss: 2.5156 - accuracy: 0.2917
Epoch 4/128
16235/16235 [==============================] - 20s 1ms/sample - loss: 2.2433 - accuracy: 0.3402
Epoch 5/128
16235/16235 [==============================] - 19s 1ms/sample - loss: 2.1052 - accuracy: 0.3908
Epoch 6/128
16235/16235 [==============================] - 19s 1ms/sample - loss: 2.0133 - accuracy: 0.4113
Epoch 7/128
16235/16235 [==============================] - 21s 1ms/sample - loss: 1.9442 - accuracy: 0.4360
Epoch 8/128
16235/16235 [==============================] - 25s 2ms/sample - loss: 1.8842 - accuracy: 0.4520
Epoch 9/128
16235/16235 [==============================] - 20s 1ms/sample - loss: 1.8306

In [278]:
# 訓練完先保存下來! 這很重要! 
#model.save("LSTM_love_letter_generator.h5")

# 預測下個字元

In [111]:
segment[0]

array(['dear darling,\ni have always wanted to write you a '],
      dtype='<U50')

In [112]:
X_test = X[0].reshape((1,50,39))

In [113]:
X[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [114]:
X_test

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]])

In [115]:
pred = model.predict(X,verbose=0)[0] #預測第1筆樣本下一個字元

這些數字是機率，all_unique_char中的字元所對應的機率，機率高的字元會被挑選做預測的下一個字

In [116]:
pred

array([9.9070106e-11, 1.3052043e-09, 6.8992222e-08, 4.6052597e-09,
       2.8658454e-07, 2.5745487e-07, 3.5795324e-06, 1.5135632e-10,
       1.2997581e-10, 1.2676900e-05, 6.9028982e-07, 2.7030837e-06,
       9.9199743e-04, 5.5679874e-09, 1.3244100e-05, 2.2519626e-09,
       1.1515544e-05, 3.9250776e-09, 7.3596368e-08, 9.0198391e-06,
       9.9755114e-01, 1.3567262e-03, 5.3423628e-07, 1.2348648e-11,
       3.6994668e-05, 3.3877807e-09, 6.3369447e-07, 2.0262206e-07,
       2.2632770e-07, 5.4214747e-11, 1.4280286e-07, 6.0345906e-06,
       1.3655009e-06, 3.7610895e-10, 2.5610980e-12, 1.3911892e-08,
       2.1594023e-08, 5.6834427e-08, 1.6078159e-09], dtype=float32)

## greedy selection + multinormial distribution
將機率縮放(開根號)，以至於常態分配比較符合我們的期望

In [117]:
pred = pred**(0.5)
pred = pred / np.sum(pred)

In [118]:
pred

array([9.04740227e-06, 3.28391470e-05, 2.38755107e-04, 6.16850157e-05,
       4.86607722e-04, 4.61214775e-04, 1.71975186e-03, 1.11828604e-05,
       1.03629618e-05, 3.23637738e-03, 7.55211106e-04, 1.49445341e-03,
       2.86291149e-02, 6.78268625e-05, 3.30798747e-03, 4.31353365e-05,
       3.08457157e-03, 5.69477743e-05, 2.46593030e-04, 2.72993301e-03,
       9.07862782e-01, 3.34810205e-02, 6.64384221e-04, 3.19420155e-06,
       5.52868936e-03, 5.29066492e-05, 7.23590027e-04, 4.09162662e-04,
       4.32435627e-04, 6.69285373e-06, 3.43495514e-04, 2.23293738e-03,
       1.06218120e-03, 1.76282592e-05, 1.45467413e-06, 1.07212574e-04,
       1.33573223e-04, 2.16699729e-04, 3.64477564e-05], dtype=float32)

基本上，會挑選機率最大的字元，但這樣會減少隨機性。
因此我們會將機率擬合成多元常態分配，從該分配中抽取字元。

In [119]:
next_onehot = np.random.multinomial(1,pred,1)
next_index = np.argmax(next_onehot)
print(next_index)
next_char_pred = all_unique_char[next_index]

20


In [178]:
print("這是第一個片段:", segment[0])
print("預測下一個字元為:",next_char_pred)
print("實際下一個字元為:",next_char[0])

這是第一個片段: ['dear darling,\ni have always wanted to write you a ']
預測下一個字元為: ['l']
實際下一個字元為: ['l']


寫成一個function吧! 接下來可是要狂用他呢!

In [24]:
def get_2d_input(text):
    all_char_in_text = []
    for j in range(segment_len):
        all_char_in_text.append(text[0][j])
    all_char_in_text = np.array(all_char_in_text)
    all_char_in_text = all_char_in_text.reshape((-1,1))
    all_char_in_text = ohe.transform(all_char_in_text).toarray()
    return all_char_in_text

def generate_next_char(text_input):
    """
    text_input (str) : 50 char
    """
    x_input = get_2d_input(np.array([text_input]))
    x_input = x_input.reshape((1,50,39)) # 1個片段，50個字元，39維字元向量
    #print(x_input)
    pred = model.predict(x_input,verbose=0)[0] #預測第1筆樣本下一個字元
    pred = pred**(0.5)
    pred = pred / (np.sum(pred)+ 0.00001)
    #print(pred)
    next_onehot = np.random.multinomial(1,pred,1)
    next_index = np.argmax(next_onehot)
    next_char_pred = all_unique_char[next_index]
    return next_char_pred

In [249]:
text_input = 't like this ,before in my life. my love for you ha'
a = generate_next_char(text_input)
print("下一個字元: ",a)

下一個字元:  ['s']


# 寫情書囉!

In [25]:
def generate_love_letter(text_input,letter_length):
    """
    text_input (str) : 50 char
    letter_length (int > 50): max number of chars in this letter(original 50 chars are included)
    output(str):
    """
    #print("初始片段\n",text_input)
    all_text_output = text_input
    while(len(all_text_output) < letter_length):
        next_char_pred = generate_next_char(text_input)
        text_input = text_input[1:] + next_char_pred[0]
        
        all_text_output = all_text_output + next_char_pred[0]
    print("生成文章\n",all_text_output)
    return all_text_output

In [26]:
begining_text = "Dear Darling,I have always wanted to write you a love letter, though I don't know how to begin."
begining_text = begining_text.lower()
my_love_letter = generate_love_letter(begining_text[:50],200)

生成文章
 dear darling,i have always wanted to write you a lode being with you. thriss you thours,
you have ogher hopet. a l ips. white upperting my diming. runtiing that the was neade eace we prepies to beca l


# 英文不好？有請谷哥翻譯

In [27]:
import googletrans
# Initial
translator = googletrans.Translator()

# Basic Translate
results = translator.translate(my_love_letter)
print(results)

Translated(src=en, dest=en, text=dear darling,i have always wanted to write you a lode being with you. thriss you thours,
you have ogher hopet. a l ips. white upperting my diming. runtiing that the was neade eace we prepies to beca l, pronunciation=dear darling,i have always wanted to write you a lode being with you. thriss you thours,
you have ogher hopet. a l ips. white upperting my diming. runtiing that the was neade eace we prepies to beca l, extra_data="{'translat...")


In [28]:
#googletrans.LANGCODES

In [29]:
print('繁體中文:\n', translator.translate(my_love_letter, dest='zh-tw').text)

繁體中文:
 親愛的親愛的，我一直想給你寫信給你。幾小時後，
你有希望。一點點白色使我昏暗。冒充我們準備過的那不整潔的面孔


# 自由創作區 7/28以後直接執行以下內容

In [30]:
from tensorflow.keras.models import load_model
model = load_model('LSTM_love_letter_generator.h5')

In [36]:
def Final_present():
    begining_text = input("請輸入英文句子(至少50個字，不能有數字):\n")
    begining_text = begining_text.lower()
    print("-------------------------------------------------------")
    my_love_letter = generate_love_letter(begining_text[:50],1000)
    print("-------------------------------------------------------")
    print('繁體中文:\n', translator.translate(my_love_letter, dest='zh-tw').text)

In [37]:
Final_present()

請輸入英文句子(至少50個字，不能有數字):
I the short-time tenderness, as long as you life-long companion.
-------------------------------------------------------
生成文章
 i the short-time tenderness, as long as you life-ly and isaver af asfout in me coonn thin'i dow ore love comy bate want to tho love.
you ralts every and kquis mat in s!opachito.
you ma tbo me thisk for ne loving you have verywiretde who connedsin toway sinders.
we bofny seepest to 
mebbingy i am shawirst’pes agains. muntit ig virm beath for.
you are the love of rucian
i, a, wontet mo,
i hasplestver love forever no one a this momentin qould brick wels i i good uxtnce that thr ow dred ted son i nowe lajuga to me.
i love you dy wlod that eving unde, i rone allayd pprsep be.
you hame you hand niver and camile ab babte wiffe.
you make you blize forsnede that is om wlounl mefeeretheal, then you have glopfest to me.
love you for very comlt on wyo me agriss my dad. i mopprest heart nas so — and known to jist nit!
i love unchac told remantin fore,